In [1]:
] activate .

  Activating project at `~/Projects/HTM/Notebooks`


In [2]:
] remove Parameters

    Updating `~/Projects/HTM/Notebooks/Project.toml`
  [d96e819e] - Parameters v0.12.3
  No Changes to `~/Projects/HTM/Notebooks/Manifest.toml`
┌ Info: We haven't cleaned this depot up for a bit, running Pkg.gc()...
└ @ Pkg /Users/julia/buildbot/worker/package_macos64/build/usr/share/julia/stdlib/v1.7/Pkg/src/Pkg.jl:639
      Active manifest files: 3 found
      Active artifact files: 78 found
      Active scratchspaces: 7 found
     Deleted no artifacts, repos, packages or scratchspaces


In [3]:
using Random

# Implementing the Numenta Temporal Memory Algorithm As Written

[Source](https://numenta.com/assets/pdf/temporal-memory-algorithm/Temporal-Memory-Algorithm-Details.pdf)

In [12]:
const NUM_COLS = 2048
const CELLS_PER_COL = 32
const ACTIVATION_THRESHOLD = 15
const INITIAL_PERMANENCE = 0.0
const CONNECTED_PERMANENCE = 0.5
const LEARNING_THRESHOLD = 12
const LEARNING_ENABLED = true
const PERMANENCE_INCREMENT = 0.05
const PERMANENCE_DECREMENT = 0.05
const PREDICTED_DECREMENT = 0.001
const SYNAPSE_SAMPLE_SIZE = 50

50

In [15]:
struct Column{T}
    cells::Set{T}
end

struct Cell{T}
    segments::Set{T}
end

struct Synapse{T}
    permanence::T
    presynaptic_cell::Cell
end

struct Segment
    cell::Cell
    synapses::Set{Synapse}
end

Cell() = Cell(Set{Segment}())
Segment(c::Cell) = Segment(c, Set{Synapse}())

function Cell(num_segments::Int)
    c = Cell()
    for i in 1:num_segments
        push!(c.segments, Segment(c))
    end
    return c
end

Column(cells_per_col, segments_per_cell) = Column(Set([Cell(segments_per_cell) for i in 1:cells_per_col]))


Column

In [6]:
struct TempMemParams
    # Constants
    num_cols::Int
    cells_per_col::Int
    
    # Keyword Args
    activation_threshold::Int
    initial_permanence::Float64
    connected_peranence::Float64
    learning_threshold::Int
    learning_enabled::Bool
    permanence_increment::Float64
    permanence_decrement::Float64
    predicted_decrement::Float64
    synapse_sample_size::Int
    initial_segments_per_cell::Int
end

In [7]:
mutable struct TempMem
    # Parameters
    ps::TempMemParams
    
    # Mutables
    columns::Array{Column, 1}
    segments::Array{Segment, 1}
    active_columns::Dict{Int, Set{Column}}
    active_cells::Dict{Int, Set{Cell}}
    winner_cells::Dict{Int, Set{Cell}}
    active_segments::Dict{Int, Set{Segment}}
    matching_segments::Dict{Int, Set{Segment}}
    num_active_potential_synapses::Dict{Int, Dict{Segment, Int}}
end

In [18]:
function TempMem(
    num_cols::Int,
    cells_per_col::Int;

    # Optional args
    activation_threshold=15,
    initial_permanence=0.3,
    connected_peranence=0.5,
    learning_threshold=12,
    learning_enabled=true,
    permanence_increment=0.05,
    permanence_decrement=0.001,
    predicted_decrement=0.05,
    synapse_sample_size=50,
    initial_segments_per_cell=0
)
    # Initialize parameter class
    ps = TempMemParams(num_cols, cells_per_col, activation_threshold, 
        initial_permanence, connected_peranence, learning_threshold,
        learning_enabled, permanence_increment, permanence_decrement,
        predicted_decrement, synapse_sample_size, initial_segments_per_cell)
    
    columns = [Column(cells_per_col, initial_segments_per_cell) for i in 1:num_cols]
    segments = [seg for col in columns for cell in col.cells for seg in cell.segments]
    

    # Past state storage
    active_columns = Dict{Int, Set{Column}}()
    active_cells = Dict{Int, Set{Cell}}()
    winner_cells = Dict{Int, Set{Cell}}()
    active_segments = Dict{Int, Set{Segment}}()
    matching_segments = Dict{Int, Set{Segment}}()
    num_active_potential_synapses = Dict{Int, Dict{Segment, Int}}()
    
    return TempMem(    
        ps,
        columns,
        segments,
        active_columns,
        active_cells, 
        winner_cells,
        active_segments,
        matching_segments,
        num_active_potential_synapses,
    )
end

Base.show(io::IO, tm::TempMem) = print(io, "TempMem\n\t$(tm.ps.num_cols) Columns")

In [19]:
tm = TempMem(
    NUM_COLS,
    CELLS_PER_COL
)

TempMem
	2048 Columns

In [21]:
function evaluate_active_cols_against_predictions!(tm::TempMem, t::Int)
    for column in tm.columns
        # Grab all segments that are active from the current column
        active_segs = segments_for_column(column, tm.active_segments[t-1])
        if column in tm.active_columns[t]
            # If the current column is an active column
            # Check if it was predicted
            if length(active_segs) > 0
                # Acivate cells corresponding to active segments
                activate_predicted_column(tm, column, active_segs, t)
            else
                # Activate all column cells
                burst_column(tm, column, active_segs, t)
            end
        elseif length(active_segs) > 0
            # If the column is not active, but it was predicted, punish it
            punish_predicted_column(column)
        end
    end
end

evaluate_active_cols_against_predictions! (generic function with 1 method)

In [22]:
function activate_predicted_column!(tm::TempMem, column, active_segs, t)
    for segment in segments_for_column(column, tm.active_segments[t-1])
        tm.active_cells[t].add(segment.cell)
        tm.winner_cells[t].add(segment.cell)
        
        if tm.learning_enabled
            for synapse in segment.synapses
                if synapse.presynaptic_cell in tm.active_cells[t-1]
                    synapse.permanence += tm.ps.permanence_increment
                else
                    synapses.permanence -= tm.ps.permanence_decrement
                end
            end
            new_synapse_count = tm.ps.synapse_sample_size - tm.num_active_potential_synapses[t-1][segment]
            grow_synapses(segment, new_synapse_count)
        end
    end
end

activate_predicted_column! (generic function with 1 method)

In [23]:
function burst_columns(tm::TempMem, column, active_segs, t)
    for cell in column.cells
        push!(tm.active_cells[t], cell)
    end
    
    if length(active_segs) > 0
        learning_segment = best_matching_segment(tm, column, active_segs, t)
        winner_cell = learning_segment.cell
    else
        winner_cell = least_used_cell(column)
        if tm.ps.learning_enabled
            learning_segment = grow_new_segment(winner_cell)
        end
    end
    
    push!(winner_cells[t], winner_cell)
    
    if tm.ps.learning_enabled
        for synapse in learning_segment.synapses
            if synapse.presynaptic_cell in tm.active_cells[t-1]
                synapse.permanence += tm.ps.permanence_increment
            else
                synapse.permanence -= tm.ps.permanence_decrement
            end
        end
        
        new_synapse_count = tm.ps.synapse_sample_size - tm.num_active_potential_synapses[t-1][learning_segment]
        grow_synapses(learning_segment, new_synapse_count)
    end
end

burst_columns (generic function with 1 method)

In [24]:
function punish_predicted_column(tm, column, active_segs, t)
    if tm.ps.learning_enabled
        for segment in active_segs
            for synapse in segment.synapses
                if synapse.presynaptic_cell in tm.active_cells[t-1]
                    synapse.permanence -= tm.ps.redicted_decrement
                end
            end
        end
    end
end

punish_predicted_column (generic function with 1 method)

In [25]:
function activate_segments(tm, t)
    for segment in tm.segments
        num_active_connected = 0
        num_active_potential = 0
        for synapse in segment.synapses
            if synapse.presynaptic_cell in tm.active_cell[t]
                if synapse.permanence >= tm.ps.connected_permanence
                    num_active_connected += 1
                end
                if synapse.permanence >= 0
                        num_active_potential += 1
                end
            end
        end
    end
    if num_active_connected >= tm.ps.activation_threshold
        push!(tm.active_segments[t], segment)
    end
    if num_active_potential >= tm.ps.learning_threshold
        push!(tm.matching_segments[t], segment)
    end
    tm.num_active_potential_synapses[t][segment] = num_active_potential
end

activate_segments (generic function with 1 method)

In [26]:
function least_used_cell(tm, column)
    fewest_segments = Inf
    for cell in column.cells
        fewest_segments = min(fewest_segments, length(cell.segments))
    end
    least_used_cells = Cell[]
    for cell in column.cells
        if cell.segments.lengh == fewest_segments
            push!(least_used_cells, cell)
        end
    end
    
    return rand(least_used_cells)
end

least_used_cell (generic function with 1 method)

In [27]:
function best_matching_segment(tm, column, active_segs, t)
    best_matching_segment = Segment()
    best_score = -1
    for segment in active_segs
        score = tm.num_active_potential_synapses[t-1][segment]
        if score > best_score
            best_matching_segment = segment
            best_score = score
        end
    end
    
    return best_matching_segment
end

best_matching_segment (generic function with 1 method)

In [29]:
function grow_synapses(tm, segment, new_synapse_count, t)
    candidates = Set([wc for wc in tm.winner_cells[t-1]])
    while (length(candidates) > 0) && (new_synapse_count > 0)
        presynaptic_cell = rand(candidates)
        pop!(candidates, presynaptic_cell)
        already_connected = false
        for synapse in segment.synapses
            if synapse.presynaptic_cell == presynaptic_cell
                already_connected = true
            end
        end
        if !already_connected
            new_synapse = create_new_synapse(segment, presynaptic_cell, tm.ps.initial_permanence)
            new_synapse_count -= 1
        end
    end
end

grow_synapses (generic function with 1 method)

In [30]:
function create_new_synapse(segment, presynaptic_cell, permanence)
    syn = Synapse(presynaptic_cell, permanence)
    push!(segment.synapses, syn)
end

function random_initial!(tm::TempMem, num_active::Int)
    active_idxs = randperm(1:tm.num_cols)[1:num_active]
    tm.active_columns[0] = tm.columns[active_idxs]
end

segments_for_column(column, segments) = filter(s -> s.cell in column.cells, segments)

segments_for_column (generic function with 1 method)